In [1]:
import processor
from data import Data, preprocess_midi_files_under
import torch
from model import load_config, load_model
import random

In [10]:
for path in ['2002', '2004', '2006', '2008', '2009', '2011', '2013', '2014', '2015', '2017', '2018']:
    preprocess_midi_files_under(f'C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/{path}', 'preprocess')

 [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\chan01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\chan02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho03.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho04.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\cho05.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\dossin01.mid] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2002\dossin02.mid] [C:/Users/Draco/Documents/GitHub/MusicTransfor

In [2]:
# Example usage:
config = load_config()
batch_size = config["batch_size"]
block_size = config["block_size"]
max_iters = config["max_iters"]
eval_interval = config["eval_interval"]
save_interval = config["save_interval"]
learning_rate = config["learning_rate"]
eval_iters = config["eval_iters"]
n_embd = config["n_embd"]
n_head = config["n_head"]
n_layer = config["n_layer"]
dropout = config["dropout"]
vocab_size = config["vocab_size"]
device = config["device"]
generated_event_length = config["generated_event_length"]

In [5]:
dataset = Data('preprocess').all_data()

torch.manual_seed(1337)

m = load_model('models/train_2.3818-val_2.3640.pth')

data = torch.tensor(Data('preprocess').all_data())
# data = torch.tensor(dataset.batch(1,2000)[2], dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def shift_sequence(sequence, rand_ints, lower_bound, upper_bound):
    # Clone the sequence to avoid modifying the original tensor
    shifted_sequence = sequence.clone()
    
    # Process each row independently
    for i in range(sequence.size(0)):
        # Define mask for elements in the specified range for this row
        mask = (sequence[i] >= lower_bound) & \
               (sequence[i] < upper_bound)
        
        # Apply the row-specific shift, clamping within 0, 127]
        shifted_sequence[i, mask] = torch.clamp(sequence[i, mask] + rand_ints[i], min=lower_bound, max=upper_bound - 1)
    
    return shifted_sequence

def multiply_sequence(sequence, rand_ints, lower_bound, upper_bound):
    # Clone the sequence to avoid modifying the original tensor
    multiplied_sequence = sequence.clone()
    
    # Process each row independently
    for i in range(sequence.size(0)):
        # Define mask for elements in the specified range for this row
        mask = (sequence[i] >= lower_bound) & \
               (sequence[i] < upper_bound)
        
        # Apply the row-specific shift, clamping within 0, 127]
        multiplied_sequence[i, mask] = torch.clamp((sequence[i, mask] - lower_bound) * rand_ints[i] + lower_bound, min=lower_bound, max=upper_bound - 1)
    
    return multiplied_sequence

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Added pitch shifting during training
    note_r_ints = [random.randint(-12, 12) for _ in range(batch_size)]
    note_lb = processor.START_IDX["notes"]
    note_ub = processor.START_IDX["notes"] + processor.RANGE_NOTES
    new_x = shift_sequence(x, note_r_ints, note_lb, note_ub)
    new_y = shift_sequence(y, note_r_ints, note_lb, note_ub)
    
    # Added velocity shifting during training
    vel_r_ints = [random.randint(-10, 10) for _ in range(batch_size)]
    vel_lb = processor.START_IDX["velocity"]
    vel_ub = processor.START_IDX["velocity"] + processor.RANGE_VEL    
    new_x = shift_sequence(new_x, vel_r_ints, vel_lb, vel_ub)
    new_y = shift_sequence(new_y, vel_r_ints, vel_lb, vel_ub)

    # Added time multiplication during training (trained on various multiplications but tending towards regular speed)
    time_r_ints = [random.randint(1, 3) for _ in range(batch_size)]
    vel_lb = processor.START_IDX["time_shift"]
    vel_ub = processor.START_IDX["time_shift"] + processor.RANGE_TIME_SHIFT  
    new_x = multiply_sequence(new_x, time_r_ints, vel_lb, vel_ub)
    new_y = multiply_sequence(new_y, time_r_ints, vel_lb, vel_ub)

    # Added length multiplication during training
    vel_lb = processor.START_IDX["length"]
    vel_ub = processor.START_IDX["length"] + processor.RANGE_LEN  
    new_x = multiply_sequence(new_x, time_r_ints, vel_lb, vel_ub)
    new_y = multiply_sequence(new_y, time_r_ints, vel_lb, vel_ub)
    
    new_x, new_y = new_x.to(device), new_y.to(device)
    return new_x, new_y

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split) 
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % config['save_interval'] == config['save_interval'] - 1:
        torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


0.908165 M parameters
step 0: train loss 2.4062, val loss 2.4250
step 100: train loss 2.4004, val loss 2.3865
step 200: train loss 2.3948, val loss 2.3898
step 300: train loss 2.3944, val loss 2.3934
step 400: train loss 2.3976, val loss 2.3876
step 500: train loss 2.3874, val loss 2.3927
step 600: train loss 2.3797, val loss 2.3883
step 700: train loss 2.3781, val loss 2.3839
step 800: train loss 2.4047, val loss 2.3867
step 900: train loss 2.4036, val loss 2.3887
step 1000: train loss 2.3777, val loss 2.3800
step 1100: train loss 2.3924, val loss 2.3893
step 1200: train loss 2.3927, val loss 2.3968
step 1300: train loss 2.3934, val loss 2.3780
step 1400: train loss 2.3890, val loss 2.3891
step 1500: train loss 2.3921, val loss 2.3928
step 1600: train loss 2.3912, val loss 2.3888
step 1700: train loss 2.3787, val loss 2.3664
step 1800: train loss 2.4029, val loss 2.3759
step 1900: train loss 2.3841, val loss 2.3827
step 2000: train loss 2.3949, val loss 2.3759
step 2100: train loss 2.

In [9]:
torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')
# m = load_model('2000_epochs.pth')

In [11]:
# generate from the model
context = torch.tensor([[306, 59, 150]], device=device)
output_sequence = m.generate(context, max_new_tokens=200)[0].tolist()

In [12]:
processor.decode_midi(output_sequence, "thing.mid")